<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/notebooks/waifu_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>Waifu Diffusion Colab</code></h2>

- 🤔 [How to use](https://github.com/NUROISEA/anime-webui-colab/wiki/Getting-started)
- 🤗 [HuggingFace Repo](https://huggingface.co/hakurei/waifu-diffusion)

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown 🧐 Select what Waifu Diffusion model you want to use

#@markdown ⚠⚠**If the cell gets stuck before getting to the WebUI, disconnect and try again.**⚠⚠ 

#@markdown A problem of colab that I cannot control, unfortunately.

#@markdown If you are using 1.4, use the **epoch 2** version
model = "1.4 epoch 2" #@param ["1.3", "1.3.5", "1.4 epoch 1", "1.4 epoch 2"]

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio.live` link to the web UI

#@markdown ----
#@markdown 💾 Save outputs to Google Drive? [What's this?](https://github.com/NUROISEA/anime-webui-colab/wiki/Saving-outputs-to-Google-Drive)
outputs_to_drive = False #@param {type:'boolean'}
output_drive_folder = "AI/Generated" #@param {type:'string'}

#@markdown ----
#@markdown 🚇 Use a different tunnel? [Confused?](https://github.com/NUROISEA/anime-webui-colab/wiki/Using-different-tunnels)
tunnel = "gradio" #@param ['gradio', 'ngrok', 'cloudflared', 'localhostrun', 'remotemoe']
#@markdown If you are using `ngrok` tunnel,
#@markdown acquire a token by following [this guide](https://github.com/NUROISEA/anime-webui-colab/wiki/Using-ngrok).
ngrok_token = "" #@param { type:'string' }
ngrok_region = "auto" #@param ["auto", "ap", "au", "eu", "in", "jp", "sa", "us"]

web_ui_folder = "/content/stable-diffusion-webui"
extensions_folder = f"{web_ui_folder}/extensions"
models_folder = f"{web_ui_folder}/models/Stable-diffusion"
vae_folder = f"{web_ui_folder}/models/VAE"
embeddings_folder = f"{web_ui_folder}/embeddings"

wd14_ui_config_dictionary = {
  "txt2img/Prompt/value": "masterpiece, best quality, high quality, absurdres, ",
  "txt2img/Negative prompt/value": "worst quality, low quality, medium quality, deleted, lowres, comic, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry"
}

wd_ui_config_dictionary = {
  "txt2img/Prompt/value": "",
  "txt2img/Negative prompt/value": ""
}

config_dictionary = {
  "CLIP_stop_at_last_layers": 1
}

model_dictionary = {
  "1.3":"https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt",
  "1.3.5":"https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-5_80000-fp32.ckpt",
  "1.4 epoch 1":"https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt",
  "1.4 epoch 2":"https://huggingface.co/saltacc/wd-1-4-anime/resolve/main/wd-1-4-epoch2-fp16.safetensors",
}

model_yaml_dictionary = {
  "1.4 epoch 1":"https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.yaml",
  "1.4 epoch 2":"https://huggingface.co/saltacc/wd-1-4-anime/resolve/main/wd-1-4-epoch2-fp16.yaml",
}

vae_link = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt"
vae_name = vae_link.split("/")[-1]

model_link = model_dictionary[model]
model_name = model_link.split("/")[-1]

git_flags = "-q"
wget_flags = "-q --show-progress"

try:
  utility
except NameError:
  !wget -q https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/utility.py -P /content/
  import utility
  utility.log_usage("waifu_diffusion")

utility.mount_drive(outputs_to_drive)

if not utility.has_run: 
  print('🚀 Installing xformers...')
  !{utility.xformers_link}

  print(f'📦 Cloning stable-diffusion-webui...')
  !git clone {git_flags} -b {utility.webui_branch} https://github.com/anime-webui-colab/stable-diffusion-webui

  for extension_link in utility.default_extensions:
    extension_name = extension_link.split('/')[-1]
    print(f'📦 Installing {extension_name}...')
    !git clone {git_flags} {extension_link} {extensions_folder}/{extension_name}
  # !git clone {git_flags} github.com/username/<repo> {extensions_folder}/<repo>

  print(f'📦 Fetching configs and embeddings...')
  for config_file in utility.default_configs:
    !wget -q {config_file} -P {web_ui_folder}/

  for embedding in utility.default_embeddings:
    !wget -q {embedding} -P {embeddings_folder}/
  # !wget {wget_flags} site.com/embedding.pt -P {embeddings_folder}/

  utility.dictionary_to_json(f"{web_ui_folder}/config.json", config_dictionary)
  
  for patch in utility.patch_list:
    !{patch}

  utility.has_run = True

utility.output_to_gdrive(outputs_to_drive, output_drive_folder)

# TODO: find a way to clean downloads of models with yaml
if model_link not in utility.models_downloaded:
  #utility.models_downloaded.append(model_link)
  !{utility.download_model(model_link)}

  if model[:3] == "1.4":
    selected_yaml = model_yaml_dictionary[model]
    !wget {wget_flags} {selected_yaml} -P {models_folder}/

!{utility.download_vae(vae_link)}

arg_list = utility.arguments(
  model=f"{models_folder}/{model_name}",
  vae=f"{vae_folder}/{vae_name}",
  tunnel=tunnel,
  ng_token=ngrok_token,
  ng_region=ngrok_region,
)

if model[:3] == "1.4":
  utility.dictionary_to_json(f"{web_ui_folder}/ui-config.json", wd14_ui_config_dictionary)
else:
  utility.dictionary_to_json(f"{web_ui_folder}/ui-config.json", wd_ui_config_dictionary)

%cd {web_ui_folder}

args = " ".join(arg_list)
!python launch.py {args}
#@markdown <i><small> Did something break? Please report it on [Github](https://github.com/NUROISEA/anime-webui-colab/issues) or DM me on [Reddit](https://www.reddit.com/user/CYTLNUROISEA)!

In [ ]:
#@title 💾 `Saving images`

#@markdown The zip file will be visible at the files tab.

#@markdown ----

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"\033[92mZipped. You can now find {archive_name} at the files tab.\033[0m")

# ----

#@markdown 📲 Copy zip to Google Drive? (Recommended)
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  utility.log_usage('zip-to-gdrive')
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"\033[92mCopied to {gdrive_folder}!\033[0m")

#@markdown ----

#@markdown 📤 Upload zip to a temporary file hosting? 
upload_to_temp = False #@param {type:'boolean'}

#@markdown 🌐 The website/host to upload to. 

#@markdown You can read more about the individual site's limits [here](https://github.com/NUROISEA/anime-webui-colab/wiki/Uploading-to-temporary-file-hosts). 
temp_file_host = "transfer.sh" #@param ["bashupload.com", "keep.sh", "temp.sh", "transfer.sh"]

host_dictionary = {
  "bashupload.com": "https://bashupload.com",
  "keep.sh": "https://free.keep.sh",
  "temp.sh": "https://temp.sh",
  "transfer.sh": f"https://transfer.sh/{archive_name}",
}
host_link = host_dictionary[temp_file_host] # not sure if colab can do this directly at {}

if upload_to_temp:
  utility.log_usage('zip-to-temp-host')
  utility.log_usage(f'zip-temp-host-{temp_file_host}')
  print(f"Uploading to {temp_file_host}...")
  !curl -T /content/{archive_name} {host_link}  
  print("\n\033[92m^^^ This is your download link! ^^^\033[0m")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = False #@param {type:'boolean'}
if delete_local_files:
  utility.log_usage('local-delete-outputs')
  %cd /content/stable-diffusion-webui
  !rm -rf outputs
  print("\033[91mDeleted the outputs folder.\033[0m")